In [ ]:
import os
import re
import sys
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
from matplotlib.offsetbox import OffsetImage, AnnotationBbox
from IPython.display import HTML, display
from pathlib import Path
from tabulate import tabulate


In [ ]:
def merge_dicts(dict1, dict2)-> dict:
    merged_dict = {}
    
    for key in dict1.keys():
        merged_dict[key] = [dict1[key], dict2[key]]
    
    return merged_dict


def nni_query(tiral_sqlite_path) -> dict:
    import sqlite3
    import json 

    table_name = 'MetricData'
    # Connect to the SQLite database
    connection = sqlite3.connect(tiral_sqlite_path)
    cursor = connection.cursor()

    query = f"""
        SELECT trialjobId, sequence, data
        FROM {table_name}
        WHERE trialjobId IN (
            SELECT trialjobId
            FROM MetricData
            WHERE type = 'FINAL'
            GROUP BY trialjobId
            ORDER BY MAX(data) DESC
            LIMIT 5
        ) 
        AND type = 'PERIODICAL'
        ORDER BY trialjobId, sequence;
    """


    # Execute the query
    cursor.execute(query)

    # Fetch the results
    results = cursor.fetchall()

    score_dict = {}
    for row in results:
        # print(f"TrialjobId: {row[0]}, Sequence: {row[1]}, Data: {row[2]}")
        trialjob_id = row[0]
        sequence = row[1]
        data = float(row[2].strip('"'))

        # If trialjob_id is not already in the dictionary, add it with an empty list
        if trialjob_id not in score_dict:
            score_dict[trialjob_id] = []

        # Append the data to the list, maintaining the order by sequence
        score_dict[trialjob_id].append(data)

    # for trialjob_id, data_list in score_dict.items():
    #     print(f"TrialjobId: {trialjob_id}, Data: {data_list}")

    query2 = f"""
    SELECT trialjobId, data
    FROM TrialJobEvent
    WHERE event = 'WAITING'
    AND trialjobId IN (SELECT trialjobId
        FROM MetricData
        WHERE type = 'FINAL'
        GROUP BY trialjobId
        ORDER BY MAX(data) DESC
        LIMIT 5);
""" 
    cursor.execute(query2)
    results2 = cursor.fetchall()
    connection.close()
    params_dict = {}
    for row in results2:
                   results = cursor.fetchall()
            print(results)
            score_dict = {}
            for row in results:
                trialjob_id = row[0]
                sequence = row[1]
                print(row[2])

            # If trialjob_id is not already in the dictionary, add it with an empty list
            if trialjob_id not in score_dict.keys():
                score_dict[trialjob_id] = []

            # Append the data to the list, maintaining the order by sequence
            score_dict[trialjob_id].append(data) #print(f"TrialjobId: {row[0]}, parameters: {row[1]}")
        trialjob_id = row[0]
        parameters = json.loads(row[1])
        # If trialjob_id is not already in the dictionary, add it with an empty list
        if trialjob_id not in params_dict:
        # Append the data to the list, maintaining the order by sequence
            params_dict[trialjob_id] = (parameters['parameters'])

    for trialjob_id, params in params_dict.items():
        print(f"TrialjobId: {trialjob_id}, Params: {params}")

    return  merge_dicts(params_dict, score_dict)   
    # Close the database connection

def show_results(path, experiment_code=None):

    if os.path.isdir(path):
            # Ottieni la lista di tutti i file e le directory nel path
            if experiment_code is None:
                dir_content = os.listdir(path)
            else:
                dir_content = [experiment_code]
            
            # Scansiona tutti gli elementi nel path
            for element in dir_content:
                print(f'experiment codename {element}')
                db_path = os.path.join(path, element,'db/nni.sqlite')
                keys = []
                if os.path.isfile(db_path):
                    experimet_best_dict = nni_query(db_path)
                    if len(experimet_best_dict) >0:
                        plt.figure(figsize=(15, 10))
                        
                        values = next(iter(experimet_best_dict.values()))
                        header = [['Trials Name'] + list(values[0].keys())]

                        for key, value in experimet_best_dict.items():
                            plt.plot(range(len(value[1])), value[1], label=f'{key}, best:{max(value[1]):.3f}')
                            keys.append(key)
                            rows = [key] +list(value[0].values())
                            header.append(rows)

                        table = tabulate(header, headers='firstrow', tablefmt='grid')
                        print(table)
                        
                        plt.legend()
                        plt.show()

                        for trial in keys: 
                            loss_trial_path = os.path.join(path, element,'trials', trial, 'Trained/loss.txt')
                            data = np.loadtxt(loss_trial_path, skiprows=1)
                            plt.figure(figsize=(15, 10))
                            plt.plot(range(len(data[:,0])), data[:,0], label='train')
                            plt.plot(range(len(data[:,1])), data[:,1], label='test')
                            plt.legend()
                            plt.title(f'{trial} loss graph')
                            plt.show()
                            image_path = os.path.join(path, element,'trials', trial, 'Trained/confusion_matrix.png')
                            img = mpimg.imread(image_path)

                            # Visualizza l'immagine
                            plt.imshow(img)
                            plt.show()                        
                            gif_path = os.path.join(path, element,'trials', trial, 'gifs')
                            gifs_names= os.listdir(gif_path)
                            if len(gifs_names) > 0 :
                                end_char = '.gif'
                                start_char = '_label'

                                gif_td = lambda gif: f'<td> <img src="{gif}" alt="Drawing" style="height: 300px;"/> </td>'
                                html = '<table>'
                                html += '<tr><td align="center"><b>Input</b></td><td><b>Output</b></td></tr>'
                                for i in range(5):
                                    pattern1 = re.compile(f'^inp{re.escape(str(i))}_.*\.gif$')
                                    pattern2 = re.compile(f'^out{re.escape(str(i))}_.*\.gif$')
                                    pattern3 = re.compile(f'{re.escape(start_char)}(.*?){re.escape(end_char)}')

                                    image_name1 =  [file for file in gifs_names if pattern1.match(file)][0]
                                    image_name2 =  [file for file in gifs_names if pattern2.match(file)][0]
                                    label = pattern3.search(image_name1).group(1)
                                    image_path1 = os.path.join(gif_path, image_name1)
                                    image_path2 = os.path.join(gif_path, image_name2)
                                    
                                    html += '<tr>'
                                    html += f'<td><b>Label: {label}</b></td>'
                                    html += gif_td(image_path1)
                                    html += gif_td(image_path2)
                                    html += '</tr>'
                                html += '</table>'
                                display(HTML(html))

                else:
                    print('no database file found in the experiment gfolder \n')               
    else:           
        print(f"given path is not a directory: {path}")



In [ ]:
#inserire qui tutte le variabili richieste per visualizzare gli esperimenti
experiment_name = "dense_alif_1"
experiment_code = 'j40hq217'
path = f'{Path.home()}/lava-dl_exp/nni_experiments/{experiment_name}/results/'

In [ ]:
show_results(path, experiment_code=experiment_code)

In [ ]:
import os
import sys
from typing import Any
import matplotlib.pyplot as plot
import numpy as np
import seaborn as sns 
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt

def gen_confusion_matrix(predictions, labels):
    
    num_label = max(labels)
    conf_matrix = confusion_matrix(predictions, labels)
    # Plot confusion matrix using Seaborn
    plt.figure(figsize=(5, 4))
    sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', cbar=False,
                xticklabels=range(num_label),
                yticklabels=range(num_label))

    plt.title('Confusion Matrix')
    plt.xlabel('Predicted label')
    plt.ylabel('True label')

    # Save the plot
    plt.show()

def custom_operation(matrix):
    # Sum the elements along the last dimension
    summed_matrix = np.sum(matrix, axis=-1)

    # Divide the sum by the number of elements in the second dimension
    divided_matrix = summed_matrix / matrix.shape[1]

    # Find the maximum value along the second dimension and its index
    max_values = np.max(divided_matrix, axis=1)
    max_indices = np.argmax(divided_matrix, axis=1)

    # Assign the index of the maximum value to the nth element
    result = np.concatenate([divided_matrix.flatten(), max_values.flatten()])
    result[-matrix.shape[0]:] = max_indices

    return max_indices

# Example usage:
n = 50  # Replace with your desired value for n
matrix = np.random.random((n, 7, 40))  # Replace this with your actual matrix
labels = np.random.randint(low=0, high=6, size=n)

result = custom_operation(matrix)
print(labels)
print(result)
gen_confusion_matrix(result, labels)
